In [333]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import json

In [285]:
doc = pd.read_csv('data/ica_data.csv', encoding='ISO-8859-1', error_bad_lines=False, delimiter=';')
doc.head()

,ID,Timestamp,Tweet,Keluhan,Respon,Bukan Keluhan/Respon,Topik Umum,Topik Spesifik,Lokasi,Waktu,Unnamed: 10
0,"6,78845E+17","21 Des 2015, 02.50.25 PM",@EL_Atheos @ridwankamil ya mungkin karena pere...,NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN
1,"6,78846E+17","21 Des 2015, 02.53.02 PM",@ridwankamil @dbmpkotabdg kang teman saya tert...,Ya,NaN,NaN,Lingkungan Hidup,pohon tumbang,Jalan Sangkuriang depan Polsek Coblong,-,NaN
2,"6,78847E+17","21 Des 2015, 02.57.55 PM",Di tribun jabar biasanya suka di post agenda k...,NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN
3,"6,78847E+17","21 Des 2015, 02.58.54 PM",@dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...,Ya,NaN,NaN,Infrastruktur,lampu penerangan jalan umum,Jalan depan Kampus LPKIA,-,NaN
4,"6,78849E+17","21 Des 2015, 03.06.49 PM","@diskamtam bapak/ibu mau tanya, kalo pemelihar...",NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN


In [303]:
data = doc.loc[:, 'Tweet':'Bukan Keluhan/Respon']
data.head()

,Tweet,Keluhan,Respon,Bukan Keluhan/Respon
0,@EL_Atheos @ridwankamil ya mungkin karena pere...,NaN,NaN,Ya
1,@ridwankamil @dbmpkotabdg kang teman saya tert...,Ya,NaN,NaN
2,Di tribun jabar biasanya suka di post agenda k...,NaN,NaN,Ya
3,@dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...,Ya,NaN,NaN
4,"@diskamtam bapak/ibu mau tanya, kalo pemelihar...",NaN,NaN,Ya


In [304]:
data = data.fillna('0')
data['Keluhan'] = data['Keluhan'].map({'Ya':1})
data['Respon'] = data['Respon'].map({'Ya':1})
data['Bukan Keluhan/Respon'] = data['Bukan Keluhan/Respon'].map({'Ya':1})

In [326]:
data_text = data.loc[:, 'Tweet']
data_tweet = data.loc[:, 'Tweet']
data_tweet[0]

'@EL_Atheos @ridwankamil ya mungkin karena perempuan bj merah yg paling kelihatan karena berada di depan, laki2 jg sering jadi bahan joke RK'

In [306]:
data_keluhan = data.loc[:, 'Keluhan']
data_keluhan[0]

nan

In [307]:
import requests
import re
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [308]:
def removeURL(sent):
    # remove username
    sent = re.sub('@[^\s]+','',sent)
    sent = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', sent, flags=re.MULTILINE)
    return sent

In [309]:
def normalization(sent):
    print(sent)
    # text normalization
    header = {
        'Content-Type' : 'application/json',
        'x-api-key' : 'PGSpcyOiDPqdlNmQ1kd66p59qhTDuLUOCqdJk4sF'
    }
    req = requests.post("https://api.prosa.ai/v1/normals", headers=header, json = { 'text' : sent})
    res = req.json()
    print(res)

In [310]:
def stemmer(sent):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(sent)

In [311]:
def tokenizer(sent):
    words = word_tokenize(sent)
    print (words)
    return words

In [314]:
data_temp = []
for tweet in data_tweet:
    tweet = removeURL(tweet)
    data_temp.append(tweet)
data_tweet = data_temp

In [315]:
# data_temp = []
# for tweet in data_tweet:
#     tweet = stemmer(tweet)
#     data_temp.append(tweet)
# data_tweet = data_temp

In [316]:
data_temp = []
for tweet in data_tweet:
    tweet = tokenizer(tweet)
    data_temp.append(tweet)
data_tweet = data_temp

['ya', 'mungkin', 'karena', 'perempuan', 'bj', 'merah', 'yg', 'paling', 'kelihatan', 'karena', 'berada', 'di', 'depan', ',', 'laki2', 'jg', 'sering', 'jadi', 'bahan', 'joke', 'RK']
['kang', 'teman', 'saya', 'tertimpa', 'pohn', 'dijln', 'sangkuriang', 'dpn', 'polsek', 'coblong', 'tlg', 'ditertibkan', 'phn', 'yg', 'sdh', 'lapuknuhun']
['Di', 'tribun', 'jabar', 'biasanya', 'suka', 'di', 'post', 'agenda', 'kang', 'emil', '.', 'Tapi', 'gak', 'setiap', 'hari', 'sih', '.', 'RT', 'Klo', 'saya', 'k', 'Bdg', 'bs', 'ketemu', 'Kang', 'dmn']
['RT', 'Lapor', 'pak', 'lampu', 'penerangan', 'jalan', 'depan', 'kampus', 'lpkia', 'ngaplek', ':', '(', ':', '(', ':', '(']
['bapak/ibu', 'mau', 'tanya', ',', 'kalo', 'pemeliharaan', 'taman2', 'yg', 'banyak', 'dibagun', 'skr', 'gimana', 'nantinya', '?']
['Pa', 'tempat', 'kerja', 'saya', 'tgl', '25', 'tetep', 'masuk', '.', 'gimana', 'itu', 'aturannya', 'pa', '?', 'kalo', 'misalnya', 'ga', 'masuk', 'dipotong', 'gaji']
['00106700466', ',', 'punten', 'sudah', 'semi

['RT', 'sampaikan', 'ke', 'pengamen', ',', 'foto', 'penertiban', 'pengamen', 'di', 'jl', '.', 'Asia', 'Afrika', 'ini', 'cc', 'https', ':', '//']
['RT', 'Punten', ',', 'air', 'di', 'rumah', 'saya', 'jalan', 'samarinda', 'no.25', ',', 'antapani', 'tdk', 'mengalir', '.', 'Mohon', 'ditindak', 'lanjuti', '...']
['kalo', 'yang', 'nakal', 'tukang', 'sampah', 'nya', 'apa', 'hukumannya', '?', 'Ini', 'di', 'halaman', 'kost', 'saya', 'saat', 'ini', '(', 'dago', '325', ')']
['RT', 'sampaikan', 'ke', 'pengamen', ',', 'foto', 'penertiban', 'pengamen', 'di', 'jl', '.', 'Asia', 'Afrika', 'ini', 'cc', 'https', ':', '//']
['RT', 'ya', 'udah', ',', 'jgn', 'pada', 'komplain', 'kalo', 'macet', 'masuk', 'lewat', 'pasteur', '.', 'nuhun', ',']
['Punten', 'mau', 'tanya', 'kalo', 'yg', 'seperti', 'ini', 'masuk', 'kategori', 'PJU', '?']
['RT', 'ya', 'udah', ',', 'jgn', 'pada', 'komplain', 'kalo', 'macet', 'masuk', 'lewat', 'pasteur', '.', 'nuhun', ',']
['RT', 'Sepertinya', 'hanya', 'tersisa', 'plang', '.', 'RT',

['Pak', 'banyak', 'mobil', 'plat', 'B', 'buang', 'sampah', 'sembaranagan', '.', 'Saya', 'tidak', 'moto', 'yg', 'buang', 'sampah', 'sebaranganan', 'karena', 'sedang', 'kagok', 'bawa', 'motor']
['Asliiiii', 'hari2', 'terakhir', 'di', 'bandung', 'masih', 'berharap', 'bertemu', 'dengan', 'kang']
['RT', '13.36', 'RT', '#', 'BreakingNews', 'Kebakaran', 'pasar', 'Kopo', 'Sayati', 'Unit', 'meluncur', 'ke', 'TKP']
['RT', 'Via', ':', 'prmpatan', 'yg', 'dkt', 'taman', 'persib', 'lampu', 'lalulintasnya', 'matii', 'cc']
['Udah', 'jumpa', 'dgn', 'sosok', 'arsitek', 'idolanya', ',', 'jangan', 'malasmalas', 'lagi', 'ya', 'fan', ',', 'kalo', 'masih', 'malas', 'udahlah', 'pulang', 'kampung', 'aja', 'sana', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?']
['knpa', 'air', 'didaerah', 'sy', 'sudah', 'lebih', 'dari', '2minggu', 'nga', 'ngalir', 'no', 'langganan', '00102280250']
['pa', 'boleh', 'usulan', 'ngak', '?', 'Ada', 'lurah', 'yang', 'aset', 'kepemilikan', 'nya', 'lebih', 'banyak'

['Selamat', 'pagi', ',', 'TL', 'simp', 'Palasari', 'Lodaya', 'sering', 'tdk', 'terlihat', 'dr', 'arah', 'Burangrang', 'krn', 'lampunya', 'mati', '&', 'amp', ';', 'tertutup', 'ranting']
['RT', 'Budaya', 'bersih', 'dan', 'buang', 'sampah', 'pd', 'tempatnta', 'adl', 'ciri', 'org', 'beradab', 'dan', 'beriman']
['RT', 'Budaya', 'bersih', 'dan', 'buang', 'sampah', 'pd', 'tempatnta', 'adl', 'ciri', 'org', 'beradab', 'dan', 'beriman']
['RT', 'Budaya', 'bersih', 'dan', 'buang', 'sampah', 'pd', 'tempatnta', 'adl', 'ciri', 'org', 'beradab', 'dan', 'beriman']
['TL', 'simp', 'Malabar-Gatsu', 'sering', 'padam', '&', 'amp', ';', 'banyak', 'lampunya', 'yg', 'mati', ',', 'mhn', 'diperhatikan', 'cc']
['RT', 'TL', 'simp', 'Malabar-Gatsu', 'sering', 'padam', '&', 'amp', ';', 'banyak', 'lampunya', 'yg', 'mati', ',', 'mhn', 'diperhatikan', 'cc']
['RT', 'Budaya', 'bersih', 'dan', 'buang', 'sampah', 'pd', 'tempatnta', 'adl', 'ciri', 'org', 'beradab', 'dan', 'beriman']
['Aslm', '...', 'Punten', 'pak', 'saya', 

['Ini', 'nomor', 'langganan', 'sya', '00602630260', 'dan', 'ini', 'nmr', 'sya', '081912621000', 'tolong', 'di', 'bantu', 'min', ',', 'terimakasih']
['RT', 'Apel', 'persiapan', '&', 'amp', ';', 'pembagian', 'tugas', 'menghadapi', 'perayaan', 'Tahun', 'Baru', '2016', 'cc', 'https', ':']
['RT', 'Apel', 'persiapan', '&', 'amp', ';', 'pembagian', 'tugas', 'menghadapi', 'perayaan', 'Tahun', 'Baru', '2016', 'cc', 'https', ':']
['RT', 'Apel', 'persiapan', '&', 'amp', ';', 'pembagian', 'tugas', 'menghadapi', 'perayaan', 'Tahun', 'Baru', '2016', 'cc', 'https', ':']
['RT', 'Apel', 'persiapan', '&', 'amp', ';', 'pembagian', 'tugas', 'menghadapi', 'perayaan', 'Tahun', 'Baru', '2016', 'cc', 'https', ':']
['kang', 'punten', 'bde', 'tumaros', 'perkawis', 'jadwal', 'mlm', 'taun', 'baruan', 'jalan', 'anu', 'd', 'tutup', 'jalan', 'mana', '?', 'Ti', 'tabuh', 'sabaraha', 'dugi', 'tabuh', 'sabaraha', '!', '!', '!']
['RT', 'Mari', 'ubah', 'pola', 'pikir', 'sampah', 'gak', 'slalu', 'dibuang', '.', 'Tapi', 'di

...

['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ',', 'siap2', 'ditewak', '.', 'Omat', '!']
['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ',', 'siap2', 'ditewak', '.', 'Omat', '!']
['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ',', 'siap2', 'ditewak', '.', 'Omat', '!']
['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ',', 'siap2', 'ditewak', '.', 'Omat', '!']
['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ',', 'siap2', 'ditewak', '.', 'Omat', '!']
['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ',', 'siap2', 'ditewak', '.', 'Omat', '!']
['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ',', 'siap2', 'ditewak', '.', 'Omat', '!']
['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ',', 'siap2', 'ditewak', '.', 'Omat', '!']
['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ',', 'siap2', 'ditewak', '.', 'Omat', '!']
['RT', 'Barudak', 'bandung', 'nu', 'sok', 'mabal', 'sakola', ','

['RT', '2', '.', 'Tdk', 'banyak', 'gd', 'parkir', 'yg', 'tersedia', ',', 'banyak', 'tempat', 'usaha', 'yg', 'tdk', 'sediakan', 'lahan', 'parkir', ',', 'lihat', 'di', 'Martadinata', 'cc']
['3', '.', 'Marka', 'jln', 'utk', 'parkir', 'perlu', 'diperbanyak', 'agr', 'warga', 'tahu', 'blh', 'parkir', 'atau', 'tdk', ',', 'termasuk', 'rambu']
['pak', 'ini', 'parkir', 'atau', 'berhenti', '?', 'Atau', 'lagi', 'jalan', 'kaki', '?']
['RT', '3', '.', 'Marka', 'jln', 'utk', 'parkir', 'perlu', 'diperbanyak', 'agr', 'warga', 'tahu', 'blh', 'parkir', 'atau', 'tdk', ',', 'termasuk', 'rambu']
['Flyer', 'dan', 'brosur', 'makin', 'banyak', 'di', '#', 'CFD', '.', 'Yuk', 'kolaborasi', 'untuk', 'kurangi', 'distribusinya', '.', 'Cc', ':']
['RT', 'Flyer', 'dan', 'brosur', 'makin', 'banyak', 'di', '#', 'CFD', '.', 'Yuk', 'kolaborasi', 'untuk', 'kurangi', 'distribusinya', '.', 'Cc', ':']
['RT', 'Flyer', 'dan', 'brosur', 'makin', 'banyak', 'di', '#', 'CFD', '.', 'Yuk', 'kolaborasi', 'untuk', 'kurangi', 'distribusi

['RT', 'Via', ':', 'Pagi2', 'pak', 'memarahi', 'warga', 'karena', 'BUANG', 'SAMPAH', 'SEMBARANGAN', '!', '#', 'plisatuhlah']
['RT', 'Via', ':', 'Pagi2', 'pak', 'memarahi', 'warga', 'karena', 'BUANG', 'SAMPAH', 'SEMBARANGAN', '!', '#', 'plisatuhlah']
['RT', 'Via', ':', 'Pagi2', 'pak', 'memarahi', 'warga', 'karena', 'BUANG', 'SAMPAH', 'SEMBARANGAN', '!', '#', 'plisatuhlah']
['RT', 'Via', ':', 'Pagi2', 'pak', 'memarahi', 'warga', 'karena', 'BUANG', 'SAMPAH', 'SEMBARANGAN', '!', '#', 'plisatuhlah']
['RT', 'Via', ':', 'Pagi2', 'pak', 'memarahi', 'warga', 'karena', 'BUANG', 'SAMPAH', 'SEMBARANGAN', '!', '#', 'plisatuhlah']
['RT', 'Via', ':', 'Pagi2', 'pak', 'memarahi', 'warga', 'karena', 'BUANG', 'SAMPAH', 'SEMBARANGAN', '!', '#', 'plisatuhlah']
['RT', 'Via', ':', 'Pagi2', 'pak', 'memarahi', 'warga', 'karena', 'BUANG', 'SAMPAH', 'SEMBARANGAN', '!', '#', 'plisatuhlah']
['RT', 'Via', ':', 'Pagi2', 'pak', 'memarahi', 'warga', 'karena', 'BUANG', 'SAMPAH', 'SEMBARANGAN', '!', '#', 'plisatuhlah']


['Kayanya', 'bukan', 'hanya', 'di', 'viaduct', 'deh', '.', 'Seueur', 'mobil', 'rongsokan', 'di', 'bahu', 'jln', '.']
['RT', 'Penertiban', 'PKL', 'Dayangsumbi', ',', 'Tamansari', '&', 'amp', ';', 'badan', 'jalan', 'Cicadas', 'hr', 'ini', '25/1/16', 'cc', 'https', ':', '/']
['jd', 'gubernur', 'jabar', 'saja', ',', 'masih', 'byk', 'yg', 'harus', 'dibereskan', ',', 'kalo', 'jkt', 'biar', 'yg', 'urus', '.']
['RT', 'Kayanya', 'bukan', 'hanya', 'di', 'viaduct', 'deh', '.', 'Seueur', 'mobil', 'rongsokan', 'di', 'bahu', 'jln', '.']
['RT', 'nanti', 'tersambung', 'dgn', 'LRT', 'Bandung', 'Raya', '.', '*apalagi', 'ditemeni', 'gebetan', ',', 'wuss', 'tdk', 'akan', 'kerasa', 'perjalanannya', '.']
['Sdh1', 'minggu', 'lebih', 'SK', 'panitia', 'pemilihan', 'RT', 'dan', 'RW', 'blm', 'jg', 'trn', 'ada', 'ap', 'selma', 'ini', 'sedngkn', 'wktu', 'SK', 'RW', 'hbis', 'tinggl', '2', 'minggu']
['Kng', 'emil', 'panitia', 'udh', 'ngadu', 'ttng', 'kpn', 'turn', 'SK', '.', 'Panitia', 'tpi', 'tdk', 'ad', 'tanggapn'

['27/1', 'URC', 'Tegallega', ',', 'banjir', 'jl', 'Leuwipanjang', 'surut']
['RT', '#', 'suaraBDG', 'via', 'Tombol', 'sama', 'lampu', 'penyebrangannya', 'engga', 'berfungsi', 'nih', ',', 'pak', 'cc', ':']
['18.49', 'Via', 'URC', 'Tegallega', ',', 'banjir', 'jl', 'Leuwipanjang', 'surut']
['RT', 'Kata', 'soal', 'kisruh', 'LGBT', ':', 'Seharusnya', 'yang', 'dibangun', 'dialog', 'bukan', 'provokasi', '.']
['RT', '#', 'suaraBDG', 'via', 'Tombol', 'sama', 'lampu', 'penyebrangannya', 'engga', 'berfungsi', 'nih', ',', 'pak', 'cc', ':']
['kang', ',', 'pju', 'jl', 'gn', 'batu-jl', 'karya', 'can', 'di', 'leresken', 'nepi', 'ayeuna', 'kang']
['RT', 'Kata', 'soal', 'kisruh', 'LGBT', ':', 'Seharusnya', 'yang', 'dibangun', 'dialog', 'bukan', 'provokasi', '.']
['RT', '#', 'lalinBDG', '18.16', ':', 'Jl', 'Moch', 'Toha', 'Banjir', 'cc', ':', '|']
['biarkan', 'pemimpin', 'seperti', 'risma', 'dan', 'ahok', 'trus', 'memimpin', 'di', 'daerahnya', 'msg2', '.', 'Mrk', 'jgn', 'diadu', '.', 'Mrk', 'contoh', 'pem

In [271]:
y = data_keluhan

In [331]:
model = Word2Vec(data_tweet, min_count=1)
model.train(data_tweet, total_examples=1, epochs=1)
x = model.wv

In [322]:
from sklearn.model_selection import train_test_split

In [327]:
text_train, text_test, y_train, y_test = train_test_split(data_text, y, test_size=0.25, random_state=42)

In [334]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

TypeError: Expected sequence or array-like, got <class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>